# 鐵達尼號生還預測：羅吉斯迴歸模型

## 1. 環境準備
導入所需的套件，並從我們建立的 `data_preprocessing.py` 腳本中導入資料準備函式。

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
def prepare_data_for_modeling(
    cleaned_data_path, 
    features_to_drop, 
    target_column, 
    test_size=0.2, 
    random_state=42):
    """
    從清理後的數據載入資料，進行特徵編碼和資料分割，為模型訓練做準備。

    Args:
        cleaned_data_path (str): train_cleaned.csv 檔案的路徑。
        features_to_drop (list): 需要從特徵集中移除的欄位列表。
        target_column (str): 目標變數的欄位名稱。
        test_size (float): 分割給測試集的資料比例。
        random_state (int): 亂數種子，確保每次分割結果一致。

    Returns:
        tuple: 回傳 (X_train, X_test, y_train, y_test)
        X_train 訓練用的特徵
        X_test  測試用的特徵
        y_train 訓練用的答案
        y_test  測試用的答案


    """
    # 載入清理後的資料
    df = pd.read_csv(cleaned_data_path)

    # --- 特徵編碼 ---
    # 對指定的類別欄位進行 One-Hot Encoding
    # drop_first=True 可以避免共線性問題
    categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'Age_Group']
    df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)
    ## df, 須處裡的 dataframe
    ## columns=categorical_features, 須處裡的欄位
    ## drop_first=True, 為了避免「共線性」（dummy variable trap），我們會捨棄每個特徵轉換後的第一個類別欄位。例如，如果Sex有'female'和'male' 兩類，get_dummies 只會產生 Sex_male 這一欄，因為當Sex_male 為 0 時，就隱含了該乘客是 'female'。
    
    # --- 定義特徵(X)與目標(y) ---
    # 定義目標變數 y [（'Survived'）- 是否生還]
    y = df_encoded[target_column]

    # 從編碼後的 df 中移除目標變數和指定的其他欄位，得到特徵集 X
    X = df_encoded.drop(columns=[target_column] + features_to_drop, axis=1)
    
    # 確保所有特徵都是數值型態
    X = X.apply(pd.to_numeric)

    # --- 資料分割 ---
    # 將資料分割為訓練集與測試集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    return X_train, X_test, y_train, y_test


## 2. 資料準備
呼叫 `prepare_data_for_modeling` 函式來載入資料、進行特徵編碼和分割，為模型訓練做好準備。

In [ ]:
# 定義清理後資料檔案的路徑和目標欄位
# 相對於 Notebook 的路徑: projects/01_Exploratory_Data_Analysis/001_EDA_Project_A_Titanic_Survival_Analysis/data/train_cleaned.csv
cleaned_data_path = '../data/train_cleaned.csv'
target_column = 'Survived'

# 這些是不具預測性的 ID 或已被轉換/合併的原始欄位
features_to_drop = ['PassengerId', 'Name', 'Ticket', 'SibSp', 'Parch', 'Fare', 'Age']

# 執行資料準備函式
X_train, X_test, y_train, y_test = prepare_data_for_modeling(
    cleaned_data_path,
    features_to_drop=features_to_drop,
    target_column=target_column
)

print("資料準備完成！")
print(f"訓練集特徵數量: {X_train.shape[1]}")
print(f"訓練集樣本數: {X_train.shape[0]}")
print(f"測試集樣本數: {X_test.shape[0]}")

## 3. 模型訓練與評估

In [ ]:
# 初始化羅吉斯迴歸模型
# max_iter 增加迭代次數以確保收斂，random_state 確保結果可重現
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# 使用訓練集進行模型訓練
log_reg.fit(X_train, y_train)

# 在測試集上進行預測
y_pred = log_reg.predict(X_test)

print("模型訓練與預測完成！")

In [ ]:
# 評估模型效能
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"模型準確率 (Accuracy): {accuracy:.4f}")
print("混淆矩陣 (Confusion Matrix):")
print(conf_matrix)
print("分類報告 (Classification Report):")
print(class_report)